In [2]:
import numpy as np
import pandas as pd
import random
import json
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import urllib
from forecastiopy import *

In [13]:
Culver = [34.021122, -118.396466]
API_KEY = '923677cb33071713bf086cd0fef8aac2'
#time = new Date().setDate(0); // lets use an arbitrary date
#unixTime = Math.floor(time.getTime()/1000);


response = urllib.request.urlopen('https://api.darksky.net/forecast/923677cb33071713bf086cd0fef8aac2/37.8267,-122.4233')
#Converts HTTPResponse object to string
string = response.read().decode('utf-8')
#Creates initial json data
json_obj = json.loads(string)
#Format json data
jsson = json.dumps(json_obj, indent = 4)
print(jsson)

#Write to json file
with open('data.json', 'w') as f:
     json.dump(jsson, f)
#fio = ForecastIO.manual('https://api.darksky.net/forecast/' + API_KEY + '/' + str(Culver[0]) + ',' + str(Culver[1]) + ',' + str(unixTime))
#print(fio.latitude)
#print(fio.timezone)
#current = FIOCurrently.FIOCurrently(fio)
#print('Temperature:', current.temperature)

{
    "daily": {
        "icon": "rain",
        "data": [
            {
                "visibility": 9.85,
                "summary": "Partly cloudy until afternoon.",
                "temperatureMax": 51.65,
                "humidity": 0.66,
                "temperatureMin": 42.83,
                "icon": "partly-cloudy-day",
                "sunriseTime": 1487861427,
                "temperatureMaxTime": 1487890800,
                "pressure": 1023.27,
                "moonPhase": 0.9,
                "apparentTemperatureMin": 37.78,
                "apparentTemperatureMax": 51.65,
                "apparentTemperatureMaxTime": 1487890800,
                "precipProbability": 0,
                "cloudCover": 0.21,
                "windSpeed": 8.53,
                "temperatureMinTime": 1487862000,
                "apparentTemperatureMinTime": 1487862000,
                "sunsetTime": 1487901509,
                "time": 1487836800,
                "windBearing": 325,
                